# Incorporação de subpalavra
:rótulo:`sec_fasttext`

Em inglês,
palavras como
"ajuda", "ajudou" e "ajudando" são
formas flexionadas da mesma palavra "help".
O relacionamento
entre "cachorro" e "cachorros"
é o mesmo que
que entre "gato" e "gatos",
e
o relacionamento
entre "menino" e "namorado"
é o mesmo que
que entre "menina" e "namorada".
Em outras línguas
como o francês e o espanhol,
muitos verbos têm mais de 40 formas flexionadas,
enquanto em finlandês,
um substantivo pode ter até 15 casos.
Em linguística,
A morfologia estuda a formação de palavras e as relações entre as palavras.
No entanto,
a estrutura interna das palavras
não foi explorado no word2vec
nem em GloVe.

## O modelo fastText

Lembre-se de como as palavras são representadas no word2vec.
Tanto no modelo skip-gram
e o modelo contínuo de saco de palavras,
diferentes formas flexionadas da mesma palavra
são representados diretamente por diferentes vetores
sem parâmetros compartilhados.
Para usar informações morfológicas,
o modelo *fastText*
propôs uma abordagem de *incorporação de subpalavras*,
onde uma subpalavra é um caractere $n$-grama :cite:`Bojanowski.Grave.Joulin.ea.2017`.
Em vez de aprender representações vetoriais em nível de palavra,
fastText pode ser considerado como
o skip-gram de nível de subpalavra,
onde cada *palavra central* é representada pela soma de
seus vetores de subpalavra.

Vamos ilustrar como obter
subpalavras para cada palavra central em fastText
usando a palavra "onde".
Primeiro, adicione os caracteres especiais “&lt;” e “&gt;”
no início e no final da palavra para distinguir prefixos e sufixos de outras subpalavras.
Em seguida, extraia os caracteres $n$-gramas da palavra.
Por exemplo, quando $n=3$,
obtemos todas as subpalavras de comprimento 3: "&lt;wh", "whe", "her", "ere", "re&gt;" e a subpalavra especial "&lt;where&gt;".


No fastText, para qualquer palavra $w$,
denotado por $\mathcal{G}_w$
a união de todas as suas subpalavras de comprimento entre 3 e 6
e sua subpalavra especial.
O vocabulário
é a união das subpalavras de todas as palavras.
Deixando $\mathbf{z}_g$
seja o vetor da subpalavra $g$ no dicionário,
o vetor $\mathbf{v}_w$ para
palavra $w$ como palavra central
no modelo skip-gram
é a soma dos seus vetores de subpalavras:

$$\mathbf{v}_w = \sum_{g\in\mathcal{G}_w} \mathbf{z}_g.$$

O resto do fastText é o mesmo que o modelo skip-gram. Comparado com o modelo skip-gram,
o vocabulário no fastText é maior,
resultando em mais parâmetros do modelo.
Além do mais,
para calcular a representação de uma palavra,
todos os seus vetores de subpalavras
tem que ser somado,
levando a uma maior complexidade computacional.
No entanto,
graças aos parâmetros compartilhados de subpalavras entre palavras com estruturas semelhantes,
palavras raras e até mesmo palavras fora do vocabulário
pode obter melhores representações vetoriais no fastText.



## Codificação de pares de bytes
:label:`codificação_de_par_de_bytes_subsec`

No fastText, todas as subpalavras extraídas precisam ter os comprimentos especificados, como $3$ a $6$, portanto, o tamanho do vocabulário não pode ser predefinido.
Para permitir subpalavras de comprimento variável em um vocabulário de tamanho fixo,
podemos aplicar um algoritmo de compressão
chamado *codificação de pares de bytes* (BPE) para extrair subpalavras :cite:`Sennrich.Haddow.Birch.2015`.

A codificação de pares de bytes realiza uma análise estatística do conjunto de dados de treinamento para descobrir símbolos comuns dentro de uma palavra,
como caracteres consecutivos de comprimento arbitrário.
A partir de símbolos de comprimento 1,
A codificação de pares de bytes mescla iterativamente o par mais frequente de símbolos consecutivos para produzir novos símbolos mais longos.
Observe que, por questões de eficiência, pares que cruzam limites de palavras não são considerados.
No final, podemos usar esses símbolos como subpalavras para segmentar palavras.
A codificação de pares de bytes e suas variantes têm sido usadas para representações de entrada em modelos populares de pré-treinamento de processamento de linguagem natural, como GPT-2 :cite:`Radford.Wu.Child.ea.2019` e RoBERTa :cite:`Liu.Ott.Goyal.ea.2019`.
A seguir, ilustraremos como funciona a codificação de pares de bytes.

Primeiro, inicializamos o vocabulário de símbolos como todos os caracteres minúsculos do inglês, um símbolo especial de fim de palavra `'_'` e um símbolo especial desconhecido `'[UNK]'`.


In [1]:
import collections

symbols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
           '_', '[UNK]']

Como não consideramos pares de símbolos que cruzam os limites das palavras,
precisamos apenas de um dicionário `raw_token_freqs` que mapeie as palavras para suas frequências (número de ocorrências)
em um conjunto de dados.
Observe que o símbolo especial `'_'` é anexado a cada palavra para que
podemos recuperar facilmente uma sequência de palavras (por exemplo, "um homem mais alto")
de uma sequência de símbolos de saída (por exemplo, "um_homem_mais_alto").
Como iniciamos o processo de mesclagem a partir de um vocabulário de apenas caracteres únicos e símbolos especiais, um espaço é inserido entre cada par de caracteres consecutivos dentro de cada palavra (chaves do dicionário `token_freqs`).
Em outras palavras, o espaço é o delimitador entre os símbolos dentro de uma palavra.


In [2]:
raw_token_freqs = {'fast_': 4, 'faster_': 3, 'tall_': 5, 'taller_': 4}
token_freqs = {}
for token, freq in raw_token_freqs.items():
    token_freqs[' '.join(list(token))] = raw_token_freqs[token]
token_freqs

{'f a s t _': 4, 'f a s t e r _': 3, 't a l l _': 5, 't a l l e r _': 4}

Definimos a seguinte função `get_max_freq_pair` que
retorna o par mais frequente de símbolos consecutivos dentro de uma palavra,
onde as palavras vêm das chaves do dicionário de entrada `token_freqs`.


In [3]:
def get_max_freq_pair(token_freqs):
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split()
        for i in range(len(symbols) - 1):
            # Key of `pairs` is a tuple of two consecutive symbols
            pairs[symbols[i], symbols[i + 1]] += freq
    return max(pairs, key=pairs.get)  # Key of `pairs` with the max value

Como uma abordagem gananciosa baseada na frequência de símbolos consecutivos,
A codificação de pares de bytes usará a seguinte função `merge_symbols` para mesclar o par mais frequente de símbolos consecutivos para produzir novos símbolos.


In [4]:
def merge_symbols(max_freq_pair, token_freqs, symbols):
    symbols.append(''.join(max_freq_pair))
    new_token_freqs = dict()
    for token, freq in token_freqs.items():
        new_token = token.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs

Agora, realizamos iterativamente o algoritmo de codificação de pares de bytes sobre as chaves do dicionário `token_freqs`. Na primeira iteração, o par mais frequente de símbolos consecutivos é `'t'` e `'a'`, portanto, a codificação de pares de bytes os mescla para produzir um novo símbolo `'ta'`. Na segunda iteração, a codificação de pares de bytes continua a mesclar `'ta'` e `'l'` para resultar em outro novo símbolo `'tal'`.


In [5]:
num_merges = 10
for i in range(num_merges):
    max_freq_pair = get_max_freq_pair(token_freqs)
    token_freqs = merge_symbols(max_freq_pair, token_freqs, symbols)
    print(f'merge #{i + 1}:', max_freq_pair)

merge #1: ('t', 'a')
merge #2: ('ta', 'l')
merge #3: ('tal', 'l')
merge #4: ('f', 'a')
merge #5: ('fa', 's')
merge #6: ('fas', 't')
merge #7: ('e', 'r')
merge #8: ('er', '_')
merge #9: ('tall', '_')
merge #10: ('fast', '_')


Após 10 iterações de codificação de pares de bytes, podemos ver que a lista `symbols` agora contém mais 10 símbolos que são mesclados iterativamente de outros símbolos.


In [6]:
print(symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '_', '[UNK]', 'ta', 'tal', 'tall', 'fa', 'fas', 'fast', 'er', 'er_', 'tall_', 'fast_']


Para o mesmo conjunto de dados especificado nas chaves do dicionário `raw_token_freqs`,
cada palavra no conjunto de dados agora é segmentada por subpalavras "fast_", "fast", "er_", "tall_" e "tall"
como resultado do algoritmo de codificação de pares de bytes.
Por exemplo, as palavras "faster_" e "taller_" são segmentadas como "fast er_" e "tall er_", respectivamente.


In [7]:
print(list(token_freqs.keys()))

['fast_', 'fast er_', 'tall_', 'tall er_']


Observe que o resultado da codificação de pares de bytes depende do conjunto de dados usado.
Também podemos usar as subpalavras aprendidas de um conjunto de dados
para segmentar palavras de outro conjunto de dados.
Como uma abordagem gananciosa, a seguinte função `segment_BPE` tenta dividir as palavras nas maiores subpalavras possíveis a partir do argumento de entrada `symbols`.


In [8]:
def segment_BPE(tokens, symbols):
    outputs = []
    for token in tokens:
        start, end = 0, len(token)
        cur_output = []
        # Segment token with the longest possible subwords from symbols
        while start < len(token) and start < end:
            if token[start: end] in symbols:
                cur_output.append(token[start: end])
                start = end
                end = len(token)
            else:
                end -= 1
        if start < len(token):
            cur_output.append('[UNK]')
        outputs.append(' '.join(cur_output))
    return outputs

A seguir, usamos as subpalavras na lista `símbolos`, que são aprendidas do conjunto de dados mencionado anteriormente,
para segmentar `tokens` que representam outro conjunto de dados.


In [9]:
tokens = ['tallest_', 'fatter_']
print(segment_BPE(tokens, symbols))

['tall e s t _', 'fa t t er_']


## Resumo

* O modelo fastText propõe uma abordagem de incorporação de subpalavras. Baseado no modelo skip-gram em word2vec, ele representa uma palavra central como a soma de seus vetores de subpalavras.
* A codificação de pares de bytes realiza uma análise estatística do conjunto de dados de treinamento para descobrir símbolos comuns dentro de uma palavra. Como uma abordagem gananciosa, a codificação de pares de bytes mescla iterativamente o par mais frequente de símbolos consecutivos.
* A incorporação de subpalavras pode melhorar a qualidade das representações de palavras raras e palavras fora do dicionário.

## Exercícios

1. Como exemplo, há cerca de $3\vezes 10^8$ $6$-gramas possíveis em inglês. Qual é o problema quando há muitas subpalavras? Como abordar o problema? Dica: consulte o final da Seção 3.2 do artigo do fastText :cite:`Bojanowski.Grave.Joulin.ea.2017`.
1. Como projetar um modelo de incorporação de subpalavras com base no modelo de saco de palavras contínuo?
1. Para obter um vocabulário de tamanho $m$, quantas operações de mesclagem são necessárias quando o tamanho do vocabulário de símbolos inicial é $n$?
1. Como estender a ideia da codificação de pares de bytes para extrair frases?


[Discussões](https://discuss.d2l.ai/t/4587)
